<a href="https://colab.research.google.com/github/ecarracedo/form-contact/blob/master/WebScraping_RNAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!pip install playwright nest_asyncio
!playwright install
!apt install -y libnss3

In [63]:
from playwright.async_api import async_playwright
import csv

async def scrape_agencias():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto("https://www.agenciasdeviajes.ar/#buscador")

        # Esperar a que cargue el buscador
        await page.wait_for_selector("input[placeholder='Ciudad o Provincia']")
        await page.fill("input[placeholder='Ciudad o Provincia']", "Córdoba")
        await page.keyboard.press("Enter")

        # Esperar a que aparezcan resultados
        await page.wait_for_selector("h3.text-lg")

        h3_agencias = await page.query_selector_all("h3.text-lg")
        print(f"🔍 Cantidad de agencias encontradas: {len(h3_agencias)}")

        agencias = []

        for h3 in h3_agencias:
            nombre = await h3.inner_text()
            telefono = ""
            correo = ""

            contenedor = await h3.evaluate_handle("node => node.parentElement.parentElement")
            contenedor_element = contenedor.as_element()

            if contenedor_element:
                parrafos = await contenedor_element.query_selector_all("p.leading-relaxed.text-sm")

                for p in parrafos:
                    texto = await p.inner_text()
                    if "Teléfono:" in texto:
                        telefono = texto.replace("Teléfono:", "").strip()
                    if "Correo electrónico:" in texto:
                        correo = texto.replace("Correo electrónico:", "").strip()

            agencias.append({
                "nombre": nombre,
                "telefono": telefono,
                "correo": correo
            })

        # Guardar en CSV
        with open("agencias_cordoba.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["nombre", "telefono", "correo"])
            writer.writeheader()
            writer.writerows(agencias)

        print("✅ Archivo generado: agencias_cordoba.csv")

# Ejecutar
await scrape_agencias()


🔍 Cantidad de agencias encontradas: 15
✅ Archivo generado: agencias_cordoba.csv


In [97]:
import asyncio
import csv
from playwright.async_api import async_playwright

async def scrapear_agencias():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        print("🌐 Cargando página...")
        await page.goto("https://www.agenciasdeviajes.ar/#buscador")

        print("⏳ Esperando el input de ciudad...")
        await page.wait_for_selector("input[placeholder*='Ciudad o Provincia']")

        print("⌨️ Escribiendo 'Córdoba' en input de ciudad...")
        await page.fill("input[placeholder*='Ciudad o Provincia']", "Córdoba")
        await page.wait_for_timeout(2000)

        print("⌛ Esperando resultados...")
        await page.wait_for_selector("h3.text-lg.text-blue-600._title-font.font-extrabold.mb-2")

        todos_los_nombres = []
        pagina = 1

        while True:
            print(f"📃 Página {pagina}: recolectando nombres...")
            nombres = await page.locator("h3.text-lg.text-blue-600._title-font.font-extrabold.mb-2").all_inner_texts()
            todos_los_nombres.extend(nombres)
            print(f"   ➕ {len(nombres)} encontrados.")

            print("🧹 Cerrando modal si está abierto...")
            await page.evaluate("""
                () => {
                    const modal = document.querySelector('[role=dialog]');
                    if (modal) {
                        window.dispatchEvent(new CustomEvent('close-modal', { detail: { id: 'video1year' }}));
                    }
                }
            """)
            await page.wait_for_timeout(1000)

            siguiente = page.locator("button[aria-label*='Siguiente']")
            if await siguiente.count() == 0 or not await siguiente.is_enabled():
                print("✅ Fin de la paginación.")
                break

            try:
                print("➡️ Haciendo clic en 'Siguiente'...")
                await siguiente.click()
                await page.wait_for_timeout(2500)
                pagina += 1
            except Exception as e:
                print(f"⚠️ Error al avanzar de página: {e}")
                break

        await browser.close()

        # Guardar en CSV
        print("💾 Guardando en agencias_cordoba.csv...")
        with open("agencias_cordoba.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Nombre de Agencia"])
            for nombre in todos_los_nombres:
                writer.writerow([nombre])

        print(f"\n✅ Total agencias guardadas: {len(todos_los_nombres)}")
        print("📁 Archivo generado: agencias_cordoba.csv")

asyncio.run(scrapear_agencias())


🌐 Cargando página...
⏳ Esperando el input de ciudad...
⌨️ Escribiendo 'Córdoba' en input de ciudad...
⌛ Esperando resultados...
📃 Página 1: recolectando nombres...
   ➕ 15 encontrados.
🧹 Cerrando modal si está abierto...


KeyboardInterrupt: 

In [89]:
import asyncio
import csv
from playwright.async_api import async_playwright

async def scrapear_agencias_completo():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        print("🌐 Cargando página...")
        await page.goto("https://www.agenciasdeviajes.ar/#buscador")

        print("⌨️ Buscando 'Córdoba'...")
        await page.wait_for_selector("input[placeholder*='Ciudad o Provincia']")
        await page.fill("input[placeholder*='Ciudad o Provincia']", "Córdoba")
        await page.wait_for_timeout(2000)

        print("⌛ Esperando resultados...")
        await page.wait_for_selector("h3.text-lg")

        agencias = []
        pagina = 1

        while True:
            print(f"📃 Página {pagina}: extrayendo agencias...")

            h3_agencias = await page.query_selector_all("h3.text-lg")
            for h3 in h3_agencias:
                nombre = await h3.inner_text()
                telefono = ""
                correo = ""

                contenedor = await h3.evaluate_handle("node => node.parentElement.parentElement")
                contenedor_element = contenedor.as_element()

                if contenedor_element:
                    parrafos = await contenedor_element.query_selector_all("p.leading-relaxed.text-sm")
                    for p in parrafos:
                        texto = await p.inner_text()
                        if "Teléfono:" in texto:
                            telefono = texto.replace("Teléfono:", "").strip()
                        if "Correo electrónico:" in texto:
                            correo = texto.replace("Correo electrónico:", "").strip()

                agencias.append({
                    "nombre": nombre,
                    "telefono": telefono,
                    "correo": correo
                })

            # Cerrar modal si está abierto
            print("🧹 Cerrando modal si está abierto...")
            await page.evaluate("""
                () => {
                    const modal = document.querySelector('[role=dialog]');
                    if (modal) {
                        window.dispatchEvent(new CustomEvent('close-modal', { detail: { id: 'video1year' }}));
                    }
                }
            """)
            await page.wait_for_timeout(1000)

            # Verificar si hay botón de siguiente
            siguiente = page.locator("button[dusk='nextPage.after']")
            if await siguiente.count() == 0 or not await siguiente.is_enabled():
                print("⛔ No hay más páginas.")
                break

            try:
                print("➡️ Haciendo clic en 'Siguiente'...")
                await siguiente.scroll_into_view_if_needed()
                await siguiente.click()
                await page.wait_for_timeout(2500)
                pagina += 1
            except Exception as e:
                print(f"⚠️ Error al hacer clic en 'Siguiente': {e}")
                break

        await browser.close()

        # Guardar en CSV
        print("💾 Guardando datos en agencias_cordoba.csv...")
        with open("agencias_cordoba.csv", "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["nombre", "telefono", "correo"])
            writer.writeheader()
            writer.writerows(agencias)

        print(f"\n✅ Total agencias guardadas: {len(agencias)}")
        print("📁 Archivo generado: agencias_cordoba.csv")

# Ejecutar en Colab
asyncio.run(scrapear_agencias_completo())


🌐 Cargando página...
⌨️ Buscando 'Córdoba'...
⌛ Esperando resultados...
📃 Página 1: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 2: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 3: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 4: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 5: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 6: extrayendo agencias...


KeyboardInterrupt: 

In [ ]:
import asyncio
import csv
import pandas as pd
from google.colab import drive
from playwright.async_api import async_playwright, TimeoutError

# Montar Google Drive
drive.mount('/content/drive')

async def scrapear_agencias_completo():
    provincia = input("📍 Ingresá la provincia que querés buscar: ").strip()

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        try:
            print("🌐 Cargando página...")
            await page.goto("https://www.agenciasdeviajes.ar/#buscador", timeout=30000)

            print(f"⌨️ Buscando '{provincia}'...")
            await page.wait_for_selector("input[placeholder*='Ciudad o Provincia']", timeout=20000)
            await page.fill("input[placeholder*='Ciudad o Provincia']", provincia)
            await page.wait_for_timeout(2000)

            print("⌛ Esperando resultados...")
            await page.wait_for_selector("h3.text-lg", timeout=20000)

            agencias = []
            pagina = 1

            while True:
                print(f"📃 Página {pagina}: extrayendo agencias...")

                h3_agencias = await page.query_selector_all("h3.text-lg")
                for h3 in h3_agencias:
                    nombre = await h3.inner_text()
                    telefono = ""
                    correo = ""
                    localidad = ""

                    contenedor = await h3.evaluate_handle("node => node.parentElement.parentElement")
                    contenedor_element = contenedor.as_element()

                    if contenedor_element:
                        parrafos = await contenedor_element.query_selector_all("p.leading-relaxed.text-sm")
                        for p in parrafos:
                            texto = await p.inner_text()
                            if "Teléfono:" in texto:
                                telefono = texto.replace("Teléfono:", "").strip()
                            if "Correo electrónico:" in texto:
                                correo = texto.replace("Correo electrónico:", "").strip()
                            if "Localidad:" in texto:
                                localidad = texto.replace("Localidad:", "").strip()

                    agencias.append({
                        "nombre": nombre,
                        "telefono": telefono,
                        "correo": correo,
                        "localidad": localidad,
                        "provincia": provincia
                    })

                # Cerrar modal si está abierto
                print("🧹 Cerrando modal si está abierto...")
                await page.evaluate("""
                    () => {
                        const modal = document.querySelector('[role=dialog]');
                        if (modal) {
                            window.dispatchEvent(new CustomEvent('close-modal', { detail: { id: 'video1year' }}));
                        }
                    }
                """)
                await page.wait_for_timeout(1000)

                # Verificar si hay botón de siguiente
                siguiente = page.locator("button[dusk='nextPage.after']")
                if await siguiente.count() == 0 or not await siguiente.is_enabled():
                    print("⛔ No hay más páginas.")
                    break

                try:
                    print("➡️ Haciendo clic en 'Siguiente'...")
                    await siguiente.scroll_into_view_if_needed()
                    await siguiente.click()
                    await page.wait_for_timeout(2500)
                    pagina += 1
                except Exception as e:
                    print(f"⚠️ Error al hacer clic en 'Siguiente': {e}")
                    break

            await browser.close()

            # Guardar CSV local
            csv_filename = f"{provincia.lower().replace(' ', '_')}_agencias_viaje.csv"
            print(f"💾 Guardando en CSV: {csv_filename}")
            with open(csv_filename, "w", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=["nombre", "telefono", "correo", "localidad", "provincia"])
                writer.writeheader()
                writer.writerows(agencias)

            # Guardar Excel en Drive
            df = pd.DataFrame(agencias)
            xlsx_path = f"/content/drive/MyDrive/{provincia.lower().replace(' ', '_')}_agencias_viaje.xlsx"
            df.to_excel(xlsx_path, index=False)
            print(f"✅ Archivo Excel guardado en Google Drive: {xlsx_path}")
            print(f"📁 Total agencias: {len(agencias)}")

        except KeyboardInterrupt:
            print("❌ Ejecución interrumpida por el usuario.")
        except TimeoutError as e:
            print(f"❌ Timeout alcanzado: {e}")
        except Exception as e:
            print(f"⚠️ Ocurrió un error inesperado: {e}")

# Ejecutar
await scrapear_agencias_completo()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📍 Ingresá la provincia que querés buscar: Entre Rios
🌐 Cargando página...
⌨️ Buscando 'Entre Rios'...
⌛ Esperando resultados...
📃 Página 1: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 2: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 3: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 4: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 5: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 6: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃 Página 7: extrayendo agencias...
🧹 Cerrando modal si está abierto...
➡️ Haciendo clic en 'Siguiente'...
📃